In [1]:
#https://github.com/MoinDalvs/Web_Scraping_Amazon_Product_Reviews/blob/main/Web_Scrapping_Amazon_Reviews.ipynb

import re
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [2]:
pages = ['https://www.amazon.com/Apple-iPhone-GSM-Unlocked-64GB/product-reviews/B0775MV9K2/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
        'https://www.amazon.com/Apple-iPhone-GSM-Unlocked-64GB/product-reviews/B0775MV9K2/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2',
        'https://www.amazon.com/Apple-iPhone-GSM-Unlocked-64GB/product-reviews/B0775MV9K2/ref=cm_cr_arp_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3']

headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
        'authority': 'www.amazon.in',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'max-age=0',
        # Requests sorts cookies= alphabetically
        # 'cookie': 'session-id=259-3113978-6678618; i18n-prefs=INR; ubid-acbin=260-8554202-6973909; lc-acbin=en_IN; csm-hit=tb:BS866TA0AKH6X86N924E+sa-7XYTQAXQHJP5ADH88228-DY27HYE0CK5V9FW24GBD|1656009294944&t:1656009294945&adb:adblk_yes; session-token=Z1j175VoYxPr2Un/9ciL3Q6lKw+QtLYYIwSQ+GLxjT06952u8vOZromD4WcFE0bs+yrUyLPy8HmIn7mTjUt8qsx3n0meC7yWKFqqwDEm5iecYedklsrNwmDrQOiaMH9lpacbdB8kgUk5IbZdg1VyhrdnY4OZrk6r350ARDEXJExuu2GZr0sV4fpbwUes/V9fDrfASeMQhVEEzmEAAHWN2g==; session-id-time=2082758401l',
        'device-memory': '8',
        'downlink': '10',
        'dpr': '0.8',
        'ect': '4g',
        'referer': 'https://www.amazon.in/OnePlus-Nord-Black-128GB-Storage/dp/B09WQY65HN/ref=sr_1_4?crid=1D99WHM86WX80&keywords=oneplus&qid=1656009113&sprefix=onep%2Caps%2C315&sr=8-4&th=1',
        'rtt': '0',
        'sec-ch-device-memory': '8',
        'sec-ch-dpr': '0.8',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-ch-viewport-width': '2400',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'service-worker-navigation-preload': 'true',
        'upgrade-insecure-requests': '1',
        'viewport-width': '2400',
    }
    

In [3]:
resp = requests.get(pages[0], headers=headers)
resp

<Response [200]>

In [4]:
soup = BeautifulSoup(resp.text, "html.parser") # html.parser, lxml

In [5]:
# 10 divs
reviews = soup.find_all("div", {"class": "a-section review aok-relative"})

In [6]:
def get_user_name(soup_obj: BeautifulSoup) -> str:
    try:
        user_name = soup_obj.find('span', {"class": "a-profile-name"}).get_text()
        return user_name
    except Exception as e:
        return 'not_found'
        print(e)

In [7]:
def get_review_stars(soup_obj: BeautifulSoup) -> str:
    try:
        review_str = soup_obj.find('a', {"class": "a-link-normal"}).get_text()
        return review_str
    except Exception as e:
        return 'not_found'
        print(e)

In [8]:
def get_review_title(soup_obj: BeautifulSoup) -> str:
    try:
        review_title = soup_obj.find('a', 
            {"class": "a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold"}
        ).get_text().strip()
        return review_title
    except Exception as e:
        return 'not_found'
        print(e)

In [9]:
def get_review_body(soup_obj: BeautifulSoup) -> str:
    try:
        review_body = soup_obj.find('span', 
            {"class": "a-size-base review-text review-text-content"}
        ).get_text().strip()
        return review_body
    except Exception as e:
        return 'not_found'
        print(e)

In [10]:
def get_review_date(soup_obj: BeautifulSoup) -> str:
    try:
        review_date = soup_obj.find('span', 
            {"class": "a-size-base a-color-secondary review-date"}
        ).get_text().strip()
        return review_date
    except Exception as e:
        return 'not_found'
        print(e)

In [11]:
def get_review_components(soup_obj: BeautifulSoup) -> dict:
    return {
        'User': get_user_name(soup_obj) , 
        'Rating': get_review_stars(soup_obj), 
        'Title': get_review_title(soup_obj),
        'Review': get_review_body(soup_obj),
        'Date': get_review_date(soup_obj)
    }

In [12]:
records = [get_review_components(rev) for rev in reviews]
df6 = pd.DataFrame.from_records(records)
#df.to_csv('amazon_reviews.csv')
df6

,User,Rating,Title,Review,Date
0,Jen piche,4.0 out of 5 stars\nGood price for 256GB iPhone\n,4.0 out of 5 stars\nGood price for 256GB iPhone,I don't really care for the iPhone 8. I don't ...,"Reviewed in the United States 🇺🇸 on April 20, ..."
1,Daleang Hong,5.0 out of 5 stars\nWorth the money!!\n,5.0 out of 5 stars\nWorth the money!!,I purchase the iPhone 8 fully unlocked that wa...,"Reviewed in the United States 🇺🇸 on June 11, 2023"
2,Kiki,"4.0 out of 5 stars\nPhone is fine, can't conne...","4.0 out of 5 stars\nPhone is fine, can't conne...",I bought this as a trap phone since my parents...,Reviewed in the United States 🇺🇸 on February 3...
3,Sarah Powell,5.0 out of 5 stars\nNo visible cosmetic damage...,5.0 out of 5 stars\nNo visible cosmetic damage...,The device was well packaged and showed no app...,"Reviewed in the United States 🇺🇸 on June 16, 2023"
4,Amazon Customer,5.0 out of 5 stars\nThat it works on another n...,5.0 out of 5 stars\nThat it works on another n...,I liked that it was as Is & had everything for...,"Reviewed in the United States 🇺🇸 on June 16, 2023"
5,Instructor Too,"5.0 out of 5 stars\nBetter Than New, It's USED...","5.0 out of 5 stars\nBetter Than New, It's USED...","""Refreshed"" is the new term for ""pre-owned,"" o...","Reviewed in the United States 🇺🇸 on April 14, ..."
6,Amazon Customer,5.0 out of 5 stars\nIPhone\n,5.0 out of 5 stars\nIPhone,The phone I ordered was in great condition no ...,"Reviewed in the United States 🇺🇸 on June 12, 2023"
7,Tim Karlak,5.0 out of 5 stars\nIt’s an Iphone 8\n,5.0 out of 5 stars\nIt’s an Iphone 8,The Iphone 8 was delivered 2 days early! The c...,"Reviewed in the United States 🇺🇸 on June 14, 2023"
8,TheReal Mani,5.0 out of 5 stars\nYes. I approve.\n,5.0 out of 5 stars\nYes. I approve.,Wow. I was so scared to order this phone becau...,"Reviewed in the United States 🇺🇸 on June 1, 2023"
9,marcia burton,4.0 out of 5 stars\nPerfect\n,4.0 out of 5 stars\nPerfect,I don’t have a picture of the phone. My mom lo...,"Reviewed in the United States 🇺🇸 on June 15, 2023"


In [13]:
ALL_REVIEWS = []

for u in pages:
    print(u)
    resp = requests.get(u, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")
    reviews = soup.find_all("div", {"class": "a-section review aok-relative"})
    records = [get_review_components(rev) for rev in reviews]
    ALL_REVIEWS.extend(records) 
    
    
out = pd.DataFrame.from_records(ALL_REVIEWS)
out

https://www.amazon.com/Apple-iPhone-GSM-Unlocked-64GB/product-reviews/B0775MV9K2/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.com/Apple-iPhone-GSM-Unlocked-64GB/product-reviews/B0775MV9K2/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.com/Apple-iPhone-GSM-Unlocked-64GB/product-reviews/B0775MV9K2/ref=cm_cr_arp_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3


,User,Rating,Title,Review,Date
0,Jen piche,4.0 out of 5 stars\nGood price for 256GB iPhone\n,4.0 out of 5 stars\nGood price for 256GB iPhone,I don't really care for the iPhone 8. I don't ...,"Reviewed in the United States on April 20, 2023"
1,Daleang Hong,5.0 out of 5 stars\nWorth the money!!\n,5.0 out of 5 stars\nWorth the money!!,I purchase the iPhone 8 fully unlocked that wa...,"Reviewed in the United States on June 11, 2023"
2,Kiki,"4.0 out of 5 stars\nPhone is fine, can't conne...","4.0 out of 5 stars\nPhone is fine, can't conne...",I bought this as a trap phone since my parents...,"Reviewed in the United States on February 3, 2023"
3,Sarah Powell,5.0 out of 5 stars\nNo visible cosmetic damage...,5.0 out of 5 stars\nNo visible cosmetic damage...,The device was well packaged and showed no app...,"Reviewed in the United States on June 16, 2023"
4,Amazon Customer,5.0 out of 5 stars\nThat it works on another n...,5.0 out of 5 stars\nThat it works on another n...,I liked that it was as Is & had everything for...,"Reviewed in the United States on June 16, 2023"
5,Instructor Too,"5.0 out of 5 stars\nBetter Than New, It's USED...","5.0 out of 5 stars\nBetter Than New, It's USED...","""Refreshed"" is the new term for ""pre-owned,"" o...","Reviewed in the United States on April 14, 2021"
6,Amazon Customer,5.0 out of 5 stars\nIPhone\n,5.0 out of 5 stars\nIPhone,The phone I ordered was in great condition no ...,"Reviewed in the United States on June 12, 2023"
7,Tim Karlak,5.0 out of 5 stars\nIt’s an Iphone 8\n,5.0 out of 5 stars\nIt’s an Iphone 8,The Iphone 8 was delivered 2 days early! The c...,"Reviewed in the United States on June 14, 2023"
8,TheReal Mani,5.0 out of 5 stars\nYes. I approve.\n,5.0 out of 5 stars\nYes. I approve.,Wow. I was so scared to order this phone becau...,"Reviewed in the United States on June 1, 2023"
9,marcia burton,4.0 out of 5 stars\nPerfect\n,4.0 out of 5 stars\nPerfect,I don’t have a picture of the phone. My mom lo...,"Reviewed in the United States on June 15, 2023"


In [14]:
def get_soup(url):
    #r = requests.get('http://localhost:8050/render.html', 
    # Run this instead if you haven't setup Splash & Docker
    r = requests.get(url, headers=headers,
    params={'url': url, 'wait': 2})
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

In [15]:
# Initialize list to store reviews data later on
reviewlist = []

# Function 2: look for web-tags in our soup, then append our data to reviewList
def get_reviews(soup):
    reviews = soup.find_all('div', {'data-hook': 'review'})
    try:
        for item in reviews:
            review = {
                'User': item.find('span', {"class": "a-profile-name"}).text.strip(),
                'Rating':  float(item.find('i', {'data-hook': 'review-star-rating'}).text.replace('out of 5 stars', '').strip()),
                'Title': item.find('a', {'data-hook': 'review-title'}).text.strip(),
                'Review': item.find('span', {'data-hook': 'review-body'}).text.strip(),
                'Date': item.find('span', {'data-hook': 'review-date'}).text.replace('Reviewed in India on', '').strip(),
            }
            reviewlist.append(review)
    except:
        pass


In [16]:
# loop through 1:x many pages, or until the css selector found only on the last page is found (when the next page button is greyed)
for x in range(1,900):
    soup = get_soup(f'https://www.amazon.com/Apple-iPhone-GSM-Unlocked-64GB/product-reviews/B0775MV9K2/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber={x}')
    print(f'Getting page: {x}')
    get_reviews(soup)
    print(len(reviewlist))

Getting page: 1
10
Getting page: 2
20
Getting page: 3
30
Getting page: 4
40
Getting page: 5
50
Getting page: 6
60
Getting page: 7
70
Getting page: 8
80
Getting page: 9
90
Getting page: 10
100
Getting page: 11
100
Getting page: 12
110
Getting page: 13
120
Getting page: 14
130
Getting page: 15
130
Getting page: 16
140
Getting page: 17
140
Getting page: 18
140
Getting page: 19
150
Getting page: 20
160
Getting page: 21
160
Getting page: 22
160
Getting page: 23
170
Getting page: 24
180
Getting page: 25
190
Getting page: 26
190
Getting page: 27
200
Getting page: 28
210
Getting page: 29
220
Getting page: 30
220
Getting page: 31
220
Getting page: 32
220
Getting page: 33
230
Getting page: 34
230
Getting page: 35
230
Getting page: 36
230
Getting page: 37
230
Getting page: 38
240
Getting page: 39
250
Getting page: 40
260
Getting page: 41
260
Getting page: 42
260
Getting page: 43
260
Getting page: 44
260
Getting page: 45
270
Getting page: 46
270
Getting page: 47
270
Getting page: 48
280
Getting pa

Getting page: 371
1980
Getting page: 372
1990
Getting page: 373
2000
Getting page: 374
2010
Getting page: 375
2010
Getting page: 376
2010
Getting page: 377
2020
Getting page: 378
2020
Getting page: 379
2030
Getting page: 380
2040
Getting page: 381
2040
Getting page: 382
2050
Getting page: 383
2050
Getting page: 384
2050
Getting page: 385
2050
Getting page: 386
2060
Getting page: 387
2070
Getting page: 388
2070
Getting page: 389
2070
Getting page: 390
2080
Getting page: 391
2080
Getting page: 392
2090
Getting page: 393
2090
Getting page: 394
2100
Getting page: 395
2110
Getting page: 396
2110
Getting page: 397
2120
Getting page: 398
2120
Getting page: 399
2120
Getting page: 400
2120
Getting page: 401
2120
Getting page: 402
2130
Getting page: 403
2140
Getting page: 404
2150
Getting page: 405
2160
Getting page: 406
2170
Getting page: 407
2170
Getting page: 408
2170
Getting page: 409
2170
Getting page: 410
2180
Getting page: 411
2180
Getting page: 412
2190
Getting page: 413
2200
Getting pag

Getting page: 728
2560
Getting page: 729
2560
Getting page: 730
2560
Getting page: 731
2560
Getting page: 732
2560
Getting page: 733
2560
Getting page: 734
2560
Getting page: 735
2560
Getting page: 736
2560
Getting page: 737
2560
Getting page: 738
2560
Getting page: 739
2560
Getting page: 740
2560
Getting page: 741
2560
Getting page: 742
2560
Getting page: 743
2560
Getting page: 744
2560
Getting page: 745
2560
Getting page: 746
2560
Getting page: 747
2560
Getting page: 748
2560
Getting page: 749
2560
Getting page: 750
2560
Getting page: 751
2560
Getting page: 752
2560
Getting page: 753
2560
Getting page: 754
2560
Getting page: 755
2560
Getting page: 756
2560
Getting page: 757
2560
Getting page: 758
2560
Getting page: 759
2560
Getting page: 760
2560
Getting page: 761
2560
Getting page: 762
2560
Getting page: 763
2560
Getting page: 764
2560
Getting page: 765
2560
Getting page: 766
2560
Getting page: 767
2560
Getting page: 768
2560
Getting page: 769
2560
Getting page: 770
2560
Getting pag

In [17]:
# Save results to a dataframe, then export as CSV
df6 = pd.DataFrame(reviewlist)
# Remove phrases until newline character in 'Title' column
df6['Title'] = df6['Title'].str.split('\n', 1).str[-1].str.strip()

# Remove "Reviewed in the United States 🇺🇸 on" from the 'Review_Date' column
# Function to extract the review date
def get_review_date(review_date: str) -> str:
    pattern = r'\w+ \d{1,2}, \d{4}'  # Regular expression pattern to match the date format
    match = re.search(pattern, review_date)
    if match:
        return match.group(0)
    return 'not_found'

# Apply the get_review_date function to the 'Review_Date' column
df6['Date'] = df6['Date'].apply(get_review_date)

# Add product name and ASIN number columns
product_name = 'Apple iPhone 8, US Version, 64GB, Space Gray - Unlocked (Renewed)'  # Replace with the actual product name
asin_number = 'B0775MV9K2'  # Replace with the actual ASIN number
df6['Product Name'] = product_name
df6['ASIN Number'] = asin_number

# Reorder the columns
df6 = df6[['Product Name', 'ASIN Number', 'User', 'Title', 'Date', 'Rating', 'Review']]

# Reorder the columns
#df1 = df1[['User', 'Title', 'Date', 'Rating', 'Review']]

df6

,Product Name,ASIN Number,User,Title,Date,Rating,Review
0,"Apple iPhone 8, US Version, 64GB, Space Gray -...",B0775MV9K2,Jen piche,Good price for 256GB iPhone,"April 20, 2023",4.0,I don't really care for the iPhone 8. I don't ...
1,"Apple iPhone 8, US Version, 64GB, Space Gray -...",B0775MV9K2,Daleang Hong,Worth the money!!,"June 11, 2023",5.0,I purchase the iPhone 8 fully unlocked that wa...
2,"Apple iPhone 8, US Version, 64GB, Space Gray -...",B0775MV9K2,Kiki,"Phone is fine, can't connect to service provider","February 3, 2023",4.0,I bought this as a trap phone since my parents...
3,"Apple iPhone 8, US Version, 64GB, Space Gray -...",B0775MV9K2,Sarah Powell,"No visible cosmetic damage, near perfect condi...","June 16, 2023",5.0,The device was well packaged and showed no app...
4,"Apple iPhone 8, US Version, 64GB, Space Gray -...",B0775MV9K2,Amazon Customer,That it works on another network,"June 16, 2023",5.0,I liked that it was as Is & had everything for...
...,...,...,...,...,...,...,...
2555,"Apple iPhone 8, US Version, 64GB, Space Gray -...",B0775MV9K2,Rudy F. Garcia,Beautiful!,"September 22, 2020",5.0,"Beautiful phone, works perfectly."
2556,"Apple iPhone 8, US Version, 64GB, Space Gray -...",B0775MV9K2,Trevor,Sent defective phone,"June 2, 2019",2.0,Sadly they had sent a phone that was defective...
2557,"Apple iPhone 8, US Version, 64GB, Space Gray -...",B0775MV9K2,Junior,Great,"February 23, 2020",5.0,"Great phone , no cracks or scuffs , battery li..."
2558,"Apple iPhone 8, US Version, 64GB, Space Gray -...",B0775MV9K2,Isabella,Camera and flashlight,"April 10, 2021",2.0,It all works but the whole back of the phone d...


In [18]:
df6.to_excel('Apple_iPhone_8.xlsx', index=False)